In [1]:
import torch
import h5py
import os
import pandas as pd

In [2]:
beams_properties_hdf5_filename = (
    "beams_properties_77faff53af5863ca146878c7c496c75e.hdf5"
)
input_dir = "output"
with h5py.File(
    os.path.join(input_dir, beams_properties_hdf5_filename), "r"
) as f:
    # print(f.keys())
    beams_properties_tensor = torch.from_numpy(f["beam_properties"][:])  # type: ignore
    beam_properties_header = f["beam_properties"].attrs["Header"]  # type: ignore
    

In [3]:
# Get the angular bin boundaries
angular_bin_boundaries = torch.arange(361) / 180 * torch.pi


In [ ]:
# load beams masks
# Define the layout index to load
layout_idx = 0

# load the beams masks for the layout
beams_masks_hdf5_filename = (
    f"beams_masks_77faff53af5863ca146878c7c496c75e_{layout_idx:03d}.hdf5"
)
input_dir = "output"
with h5py.File(
    os.path.join(input_dir, beams_masks_hdf5_filename), "r"
) as beams_masks_hdf5:
    beams_masks = torch.from_numpy(beams_masks_hdf5["beam_mask"][:]) # type: ignore
layout_beams_properties = beams_properties_tensor[beams_properties_tensor[:, 0] == layout_idx].clone()
layout_beams_properties[:,2] +=1
# Digitize the angles
digitized_angles = torch.bucketize(
    layout_beams_properties[:, 3], angular_bin_boundaries, right=True
)
layout_beams_properties[:, 3] = digitized_angles

columns_name: list = beam_properties_header.tolist()  # type: ignore
columns_name[3] = "angle_bin"
layout_beams_property_df = pd.DataFrame(
		layout_beams_properties.numpy(),
		columns=columns_name,  # type: ignore
)

/tmp/ipykernel_53293/277874979.py:17: UserWarning: torch.searchsorted(): input value tensor is non-contiguous, this will lower the performance due to extra data copy when converting non-contiguous tensor to contiguous, please use contiguous input value tensor if possible. This message will only appear once per program. (Triggered internally at /pytorch/aten/src/ATen/native/BucketizationUtils.h:32.)
  digitized_angles = torch.bucketize(


In [ ]:
# filter out the beams with FWHM > 4
layout_beams_property_df_filtered_by_fwhm = layout_beams_property_df[
		layout_beams_property_df["FWHM (mm)"] < 4
]

layout_beams_property_tensor_filtered_by_fwhm = layout_beams_properties[layout_beams_properties[:,4] < 4]


torch.Size([3003, 11])


In [12]:
# For each detector unit, mask shape = (512*512,)
# if we fix the detector unit id, beams_properties shape = (N, 11)
# pixel_beam_property = layout_beams_property_tensor_filtered_by_fwhm[]
detector_unit_id = 0
detector_unit_beams_properties = layout_beams_properties[
		layout_beams_properties[:, 1] == detector_unit_id
].clone()
detector_unit_beams_properties_df = pd.DataFrame(
		detector_unit_beams_properties.numpy(),
		columns=columns_name,  # type: ignore
)
beams_masks[detector_unit_id].unique()

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [27]:
# For a single detector unit, each pixel acquire an angular bin increment or not based on the beams mask

# for px_id in torch.arange(512*512):
	# get the mask for the pixel
	# pixel_beam_id = beams_masks[detector_unit_id][px_id]
beams_mask_detector_unit = beams_masks[detector_unit_id].clone()
beams_mask_detector_unit[beams_mask_detector_unit == 0] = torch.nan
detector_unit_beams_properties = detector_unit_beams_properties[torch.argsort(detector_unit_beams_properties[:, 2])]

# print(beams_ids_origin.shape)
print(detector_unit_beams_properties[:,2])


tensor([1., 2., 3., 4., 5., 6., 7., 8., 9.])


In [17]:
a = torch.tensor([[1, 2, 3], [1, 2, 3], [1, 2, 3], [3, 1, 2]])
print(a.shape)
sort_idx = torch.argsort(a[3, :], descending=True)
print(sort_idx)
print(a[:,sort_idx])

torch.Size([4, 3])
tensor([0, 2, 1])
tensor([[1, 3, 2],
        [1, 3, 2],
        [1, 3, 2],
        [3, 2, 1]])
